# Permissions API Demo

This notebook demonstrates the Permissions API functionality including:
- Creating permissions, groups, and users
- Setting permissions at different levels
- Viewing calculated permissions
- Debugging permission resolution chains
- Viewing audit history

**Prerequisites:** Start the API with `dotnet run --project ../src/PermissionsApi`

In [1]:
import requests
import json
from datetime import datetime

BASE_URL = "http://localhost:5000/api/v1"

def pretty_print(response):
    print(f"Status: {response.status_code}")
    if response.text:
        print(json.dumps(response.json(), indent=2))
    print()

## 1. Create Permissions

In [21]:
# Create permissions with different default settings
permissions = [
    {"name": "read", "description": "Read access to resources", "isDefault": True},
    {"name": "write", "description": "Write access to resources", "isDefault": False},
    {"name": "delete", "description": "Delete resources", "isDefault": False},
    {"name": "admin", "description": "Administrative access", "isDefault": False}
]

for perm in permissions:
    response = requests.post(f"{BASE_URL}/permissions", json=perm)
    pretty_print(response)

Status: 201
{
  "name": "read",
  "description": "Read access to resources",
  "isDefault": true,
  "id": "read"
}

Status: 201
{
  "name": "write",
  "description": "Write access to resources",
  "isDefault": false,
  "id": "write"
}

Status: 201
{
  "name": "delete",
  "description": "Delete resources",
  "isDefault": false,
  "id": "delete"
}

Status: 201
{
  "name": "admin",
  "description": "Administrative access",
  "isDefault": false,
  "id": "admin"
}



In [4]:
# List all permissions
response = requests.get(f"{BASE_URL}/permissions")
pretty_print(response)

Status: 200
[
  {
    "name": "delete",
    "description": "Delete resources",
    "isDefault": false,
    "id": "delete"
  },
  {
    "name": "admin",
    "description": "Administrative access",
    "isDefault": false,
    "id": "admin"
  },
  {
    "name": "write",
    "description": "Write access to resources",
    "isDefault": false,
    "id": "write"
  },
  {
    "name": "read",
    "description": "Read access to resources",
    "isDefault": true,
    "id": "read"
  }
]



## 2. Create Groups

In [5]:
# Create groups
editors_response = requests.post(f"{BASE_URL}/groups", json={"name": "editors"})
pretty_print(editors_response)
editors_id = editors_response.json()["id"]

admins_response = requests.post(f"{BASE_URL}/groups", json={"name": "admins"})
pretty_print(admins_response)
admins_id = admins_response.json()["id"]

Status: 201
{
  "id": "11b115e5-906c-435d-ad4b-5cee751dd0a5",
  "name": "editors"
}

Status: 201
{
  "id": "612d3d1b-8ad0-4b39-bad3-0bdfaddc6bb2",
  "name": "admins"
}



In [6]:
# Set permissions for editors group
response = requests.put(
    f"{BASE_URL}/groups/{editors_id}/permissions",
    json={
        "allow": ["write"],
        "deny": ["delete"],
        "principal": "admin@example.com",
        "reason": "Standard editor permissions"
    }
)
pretty_print(response)

Status: 200



In [7]:
# Set permissions for admins group
response = requests.put(
    f"{BASE_URL}/groups/{admins_id}/permissions",
    json={
        "allow": ["write", "delete", "admin"],
        "principal": "admin@example.com",
        "reason": "Full admin access"
    }
)
pretty_print(response)

Status: 200



## 3. Create Users

In [8]:
# Create user in editors group
response = requests.post(
    f"{BASE_URL}/users",
    json={
        "email": "alice@example.com",
        "groups": [editors_id]
    }
)
pretty_print(response)

Status: 201



In [9]:
# Create user in admins group
response = requests.post(
    f"{BASE_URL}/users",
    json={
        "email": "bob@example.com",
        "groups": [admins_id]
    }
)
pretty_print(response)

Status: 201



## 4. View Calculated Permissions

In [13]:
# Alice's permissions (editor)
response = requests.get(f"{BASE_URL}/users/alice@example.com/permissions")
pretty_print(response)

Status: 200
{
  "email": "alice@example.com",
  "allow": [
    "read",
    "write"
  ],
  "deny": [
    "delete"
  ]
}



In [14]:
# Bob's permissions (admin)
response = requests.get(f"{BASE_URL}/users/bob@example.com/permissions")
pretty_print(response)

Status: 200
{
  "email": "bob@example.com",
  "allow": [
    "admin",
    "delete",
    "read",
    "write"
  ],
  "deny": []
}



## 5. User-Level Permission Override

In [15]:
# Give Alice delete permission (override group DENY)
response = requests.put(
    f"{BASE_URL}/users/alice@example.com/permissions/delete",
    json={
        "access": "ALLOW",
        "principal": "admin@example.com",
        "reason": "Special exception for Alice"
    }
)
pretty_print(response)

Status: 200



In [16]:
# Check Alice's permissions again
response = requests.get(f"{BASE_URL}/users/alice@example.com/permissions")
pretty_print(response)

Status: 200
{
  "email": "alice@example.com",
  "allow": [
    "delete",
    "read",
    "write"
  ],
  "deny": []
}



## 6. Debug Permission Resolution

In [17]:
# Debug Alice's permission resolution chain
response = requests.get(f"{BASE_URL}/user/alice@example.com/debug")
pretty_print(response)

Status: 200
{
  "email": "alice@example.com",
  "permissions": [
    {
      "permission": "delete",
      "finalResult": "ALLOW",
      "chain": [
        {
          "level": "Default",
          "source": "system",
          "action": "NONE"
        },
        {
          "level": "Group",
          "source": "editors",
          "action": "DENY"
        },
        {
          "level": "User",
          "source": "alice@example.com",
          "action": "ALLOW"
        }
      ]
    },
    {
      "permission": "read",
      "finalResult": "ALLOW",
      "chain": [
        {
          "level": "Default",
          "source": "system",
          "action": "ALLOW"
        }
      ]
    },
    {
      "permission": "write",
      "finalResult": "ALLOW",
      "chain": [
        {
          "level": "Default",
          "source": "system",
          "action": "NONE"
        },
        {
          "level": "Group",
          "source": "editors",
          "action": "ALLOW"
        }
     

## 7. Audit History

In [18]:
# View global history
response = requests.get(f"{BASE_URL}/history?skip=0&count=10")
pretty_print(response)

Status: 200
[
  {
    "timestampUtc": "2025-10-25T00:52:41.115737",
    "changeType": "CREATE",
    "entityType": "User",
    "entityId": "bob@example.com",
    "entityAfterChange": {
      "$type": "user",
      "email": "bob@example.com",
      "groups": [
        "612d3d1b-8ad0-4b39-bad3-0bdfaddc6bb2"
      ],
      "permissions": {},
      "id": "bob@example.com"
    },
    "principal": null,
    "reason": null,
    "id": "d837753f-7d23-4552-ac8e-e81d6a8d4d88"
  },
  {
    "timestampUtc": "2025-10-25T00:52:40.581199",
    "changeType": "CREATE",
    "entityType": "User",
    "entityId": "alice@example.com",
    "entityAfterChange": {
      "$type": "user",
      "email": "alice@example.com",
      "groups": [
        "11b115e5-906c-435d-ad4b-5cee751dd0a5"
      ],
      "permissions": {
        "delete": "ALLOW"
      },
      "id": "alice@example.com"
    },
    "principal": null,
    "reason": null,
    "id": "35d36c93-0970-4bba-867a-bd6e1dcd3465"
  },
  {
    "timestampUtc": "20

In [22]:
# View Alice's history
response = requests.get(f"{BASE_URL}/users/alice@example.com/history")
pretty_print(response)

Status: 200
[
  {
    "timestampUtc": "2025-10-25T00:52:40.581199",
    "changeType": "CREATE",
    "entityType": "User",
    "entityId": "alice@example.com",
    "entityAfterChange": {
      "$type": "user",
      "email": "alice@example.com",
      "groups": [
        "11b115e5-906c-435d-ad4b-5cee751dd0a5"
      ],
      "permissions": {
        "delete": "ALLOW"
      },
      "id": "alice@example.com"
    },
    "principal": null,
    "reason": null,
    "id": "35d36c93-0970-4bba-867a-bd6e1dcd3465"
  }
]



In [23]:
# View editors group history
response = requests.get(f"{BASE_URL}/groups/{editors_id}/history")
pretty_print(response)

Status: 200
[
  {
    "timestampUtc": "2025-10-25T00:52:38.848813",
    "changeType": "UPDATE",
    "entityType": "Group",
    "entityId": "11b115e5-906c-435d-ad4b-5cee751dd0a5",
    "entityAfterChange": {
      "$type": "group",
      "id": "11b115e5-906c-435d-ad4b-5cee751dd0a5",
      "name": "editors",
      "permissions": {
        "write": "ALLOW",
        "delete": "DENY"
      }
    },
    "principal": "admin@example.com",
    "reason": "Standard editor permissions",
    "id": "27bb079b-88f1-4d35-aa7c-1c0b1be2761e"
  },
  {
    "timestampUtc": "2025-10-25T00:52:38.02306",
    "changeType": "CREATE",
    "entityType": "Group",
    "entityId": "11b115e5-906c-435d-ad4b-5cee751dd0a5",
    "entityAfterChange": {
      "$type": "group",
      "id": "11b115e5-906c-435d-ad4b-5cee751dd0a5",
      "name": "editors",
      "permissions": {}
    },
    "principal": null,
    "reason": null,
    "id": "2d46341b-4441-4cc9-b771-f3441564c29b"
  }
]



## 8. Batch Operations

In [24]:
# Create user with multiple groups
response = requests.post(
    f"{BASE_URL}/users",
    json={
        "email": "charlie@example.com",
        "groups": [editors_id, admins_id]
    }
)
pretty_print(response)

Status: 201



In [25]:
# Set multiple user permissions at once
response = requests.put(
    f"{BASE_URL}/users/charlie@example.com/permissions",
    json={
        "allow": ["admin"],
        "deny": ["delete"],
        "principal": "admin@example.com",
        "reason": "Custom permission set for Charlie"
    }
)
pretty_print(response)

Status: 200



In [26]:
# Check Charlie's final permissions (groups are processed alphabetically)
response = requests.get(f"{BASE_URL}/users/charlie@example.com/permissions")
pretty_print(response)

Status: 200
{
  "email": "charlie@example.com",
  "allow": [
    "admin",
    "read",
    "write"
  ],
  "deny": [
    "delete"
  ]
}



In [27]:
# Debug Charlie's resolution to see group ordering
response = requests.get(f"{BASE_URL}/user/charlie@example.com/debug")
pretty_print(response)

Status: 200
{
  "email": "charlie@example.com",
  "permissions": [
    {
      "permission": "admin",
      "finalResult": "ALLOW",
      "chain": [
        {
          "level": "Default",
          "source": "system",
          "action": "NONE"
        },
        {
          "level": "Group",
          "source": "admins",
          "action": "ALLOW"
        },
        {
          "level": "User",
          "source": "charlie@example.com",
          "action": "ALLOW"
        }
      ]
    },
    {
      "permission": "delete",
      "finalResult": "DENY",
      "chain": [
        {
          "level": "Default",
          "source": "system",
          "action": "NONE"
        },
        {
          "level": "Group",
          "source": "admins",
          "action": "ALLOW"
        },
        {
          "level": "Group",
          "source": "editors",
          "action": "DENY"
        },
        {
          "level": "User",
          "source": "charlie@example.com",
          "action":

## 9. Toggle Default Permission

In [28]:
# Toggle read permission default status
response = requests.put(
    f"{BASE_URL}/permissions/read/default",
    json={
        "principal": "admin@example.com",
        "reason": "Testing default toggle"
    }
)
pretty_print(response)

Status: 400
{
  "type": "https://tools.ietf.org/html/rfc9110#section-15.5.1",
  "title": "One or more validation errors occurred.",
  "status": 400,
  "errors": {
    "$": [
      "The JSON value could not be converted to System.Boolean. Path: $ | LineNumber: 0 | BytePositionInLine: 1."
    ]
  },
  "traceId": "00-5d34064d719b43fb620b7d4ff87e55ae-7786a3a56f845f93-00"
}



In [29]:
# Create a new user with no groups to see default permissions
response = requests.post(
    f"{BASE_URL}/users",
    json={"email": "dave@example.com", "groups": []}
)
pretty_print(response)

response = requests.get(f"{BASE_URL}/users/dave@example.com/permissions")
pretty_print(response)

Status: 201

Status: 200
{
  "email": "dave@example.com",
  "allow": [
    "read"
  ],
  "deny": []
}



## 10. Cleanup (Optional)

In [30]:
# Delete users
for email in ["alice@example.com", "bob@example.com", "charlie@example.com", "dave@example.com"]:
    response = requests.delete(f"{BASE_URL}/users/{email}")
    print(f"Delete {email}: {response.status_code}")

# Delete groups
for group_id in [editors_id, admins_id]:
    response = requests.delete(f"{BASE_URL}/groups/{group_id}")
    print(f"Delete group {group_id}: {response.status_code}")

# Delete permissions
for perm in ["read", "write", "delete", "admin"]:
    response = requests.delete(f"{BASE_URL}/permissions/{perm}")
    print(f"Delete permission {perm}: {response.status_code}")

Delete alice@example.com: 204
Delete bob@example.com: 204
Delete charlie@example.com: 204
Delete dave@example.com: 204
Delete group 11b115e5-906c-435d-ad4b-5cee751dd0a5: 204
Delete group 612d3d1b-8ad0-4b39-bad3-0bdfaddc6bb2: 204
Delete permission read: 204
Delete permission write: 204
Delete permission delete: 204
Delete permission admin: 204
